<a href="https://colab.research.google.com/github/sagiodev/stablediffusion_webui/blob/master/StableDiffusionUI_ngrok_sagiodev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [stable-diffusion-art.com](https://stable-diffusion-art.com) - Check out latest tutorials.

### [Guide to use this notebook](https://stable-diffusion-art.com/automatic1111-colab/) - Leave comment if you have questions.

Latest version of this notebook can be found at [here](https://colab.research.google.com/github/sagiodev/stablediffusion_webui/blob/master/StableDiffusionUI_ngrok_sagiodev.ipynb).

This notebook launches [AUTOMATIC1111 Stable Diffusion Web UI](https://github.com/AUTOMATIC1111/stable-diffusion-webui)


## Instruction
In the next cell...
1. Review username and password.
2. Check the models you want to load.
3. Click the Play button on left to start.
4. Follow the `gradio.live` or `ngrok.io` link to start using AUTOMATIC1111.

## ngrok (Optional)
Use [ngrok](https://ngrok.com/) if you experience GUI issues like unresponsive button after generating images
  1. Go to https://ngrok.com/
  2. create an account
  3. verify email
  4. Copy the authoken from https://dashboard.ngrok.com/get-started/your-authtoken and paste in ngrok field below.

## Update
- 4/12/2023 Resolve A1111 crash when changing model in free version.
- 4/5/2023 down A1111 version for stability
- 4/3/2023 Disabled xformer because error, up A1111 version
- 3/28/2023 bug fix colab update




In [ ]:
import os, subprocess, time, glob
#@title <font size="6" color="orange">Stable diffusion</font>
#@markdown #### Follow the URL link **`https://xxxxxxx.gradio.live`** to launch the app.
#@markdown ##### (**`https://xxxxx.ngrok.io`** if use ngrok.)
output_path = 'AI_PICS' #@param {type:"string"}
username='a' #@param {type:"string"}
password='a' #@param {type:"string"}
NGROK ='' #@param {type: "string"}
#@markdown Save images, settings, Lora models, embeddings, load models in Google Drive.

Save_In_Google_Drive = 'Small models, images and settings' #@param ['Everything', 'Small models, images and settings', "Nothing"]
class GDriveSaveMode:
    Everything = 1
    Models_only = 2
    Nothing = 3
if Save_In_Google_Drive == 'Small models, images and settings':
  gMode = GDriveSaveMode.Models_only
elif Save_In_Google_Drive == 'Everything':
  gMode = GDriveSaveMode.Everything
elif Save_In_Google_Drive == 'Nothing':
  gMode = GDriveSaveMode.Nothing
else:
  raise ValueError("Save_In_Google_Drive value invalid.")

Clear_Log = True #@param{type:'boolean'}
#@markdown ## Only check the models you are going to use:
#@markdown ### v1 models:
v1_4_model = False #@param{type: "boolean"}
v1_5_model = True #@param{type: "boolean"}
v1_5_inpainting_model = False #@param{type: "boolean"}
F222_model = False #@param{type: "boolean"}
DreamShaper_model = False #@param{type: "boolean"}
OpenJourney_model = False #@param{type: "boolean"}
Anything_v3_model = False #@param{type: "boolean"}
Inkpunk_Diffusion_model = False #@param{type: "boolean"}
instruct_pix2pix_model = False #@param{type: "boolean"}

#@markdown ### v2 models:
v2_1_768_model = False #@param{type: "boolean"}
v2_1_512_model = False #@param{type: "boolean"}
v2_depth_model = False #@param{type: "boolean"}

#@markdown ### Extensions:
ControlNet = False #@param{type: "boolean"}

#@markdown ### Install models from URL (separate them with comma). Optionally save copies in `output_path`/models
Model_from_URL = '' #@param {type: "string"}
Save_model_in_Google_Drive = False #@param{type: "boolean"}


if gMode != GDriveSaveMode.Nothing:
  # connect to google drive
  from google.colab import drive
  drive.mount('/content/drive') 
  output_path = '/content/drive/MyDrive/' + output_path
  if gMode == GDriveSaveMode.Everything:
    # install A1111 in AI_PIC
    root = output_path 
  elif gMode == GDriveSaveMode.Models_only:
    # install A1111 in Colab
    root = '/content/' 
    # make necessary directories if not exists
    !mkdir -p {output_path}/outputs
    !mkdir -p {output_path}/models
    !mkdir -p {output_path}/ESRGAN
    !mkdir -p {output_path}/hypernetworks
  else:
    raise ValueError("Unexpected gMode: %s"%gMode)
else:
  # Don't connect to google drive
  output_path = '/content/' + output_path
  root = '/content/' 


def clear():
    from IPython.display import clear_output; return clear_output()

def fetch_bytes(url_or_path):
    if str(url_or_path).startswith('http://') or str(url_or_path).startswith('https://'):
        from urllib.request import urlopen 
        return urlopen(url_or_path) 
    return open(url_or_path, 'r')

def packages():
    import sys, subprocess
    return [r.decode().split('==')[0] for r in subprocess.check_output([sys.executable, '-m', 'pip', 'freeze']).split()]

def downloadModel(url):
  if 'huggingface.co' in url:
    filename = url.split('/')[-1]
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
  else:
    # civitai
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}

def download_model():
  %cd {root}/stable-diffusion-webui/models/Stable-diffusion
  print('⏳ Downloading models ...')
  if v1_4_model:
    downloadModel('https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt')
  if v1_5_model:  
    downloadModel('https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt')
  if v1_5_inpainting_model:  
    downloadModel('https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt')
  if F222_model:
    downloadModel('https://huggingface.co/acheong08/f222/resolve/main/f222.ckpt')
  if DreamShaper_model:
    downloadModel('https://huggingface.co/Lykon/DreamShaper/resolve/main/Dreamshaper_3.32_baked_vae_clip_fix_half.ckpt')
  if OpenJourney_model:
    downloadModel('https://huggingface.co/prompthero/openjourney/resolve/main/mdjrny-v4.ckpt')
  if Anything_v3_model:
    downloadModel('https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors')
  if Inkpunk_Diffusion_model:
    downloadModel('https://huggingface.co/Envvi/Inkpunk-Diffusion/resolve/main/Inkpunk-Diffusion-v2.ckpt')
  if instruct_pix2pix_model:
    downloadModel('https://huggingface.co/timbrooks/instruct-pix2pix/resolve/main/instruct-pix2pix-00-22000.ckpt')

  if v2_1_768_model:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt')
    !wget -nc https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml -O {root}/stable-diffusion-webui/models/Stable-diffusion/v2-1_768-ema-pruned.yaml

  if v2_1_512_model:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.ckpt')
    !wget -nc https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference.yaml -O {root}/stable-diffusion-webui/models/Stable-diffusion/v2-1_512-ema-pruned.yaml

  if v2_depth_model:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-2-depth/resolve/main/512-depth-ema.ckpt')
    !wget -nc https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-midas-inference.yaml -O {root}/stable-diffusion-webui/models/Stable-diffusion/512-depth-ema.yaml

  if Model_from_URL:
      for m in Model_from_URL.split(','):
        %cd {root}/stable-diffusion-webui/models/Stable-diffusion
        downloadModel(m)
        if Save_model_in_Google_Drive and gMode == GDriveSaveMode.Models_only:
          %cd {output_path}/models
          downloadModel(m)

  # download VAEs
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-ema-original/resolve/main/vae-ft-ema-560000-ema-pruned.ckpt -d {root}/stable-diffusion-webui/models/VAE/ -o vae-ft-ema-560000-ema-pruned.ckpt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {root}/stable-diffusion-webui/models/VAE/ -o vae-ft-mse-840000-ema-pruned.ckpt

def installxformers():
  #!pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl
  #%pip install --no-deps -q https://github.com/brian6091/xformers-wheels/releases/download/0.0.15.dev0%2B4c06c79/xformers-0.0.15.dev0+4c06c79.d20221205-cp38-cp38-linux_x86_64.whl
  %pip install -q xformers

def updatePython():
  #install python 3.10
  !apt-get update -y
  !apt-get install python3.10

  #change alternatives
  !rm /usr/local/bin/python
  !rm /usr/local/bin/pip
  !sudo apt-get install python3.10-distutils
  !sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 2
  !wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py

updatePython()


!mkdir -p {root}
os.chdir(root)
!apt-get -y install -qq aria2
!pip install pyngrok
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
download_model()
#installxformers()
%cd {root}/stable-diffusion-webui
!git checkout -f a9fed7c364061ae6efb37f797b6b522cb3cf7aa2
#!git checkout -f 22bcc7be428c94e9408f589966c2040187245d81
!git config core.filemode false
#!COMMANDLINE_ARGS="--exit" REQS_FILE="requirements.txt" python launch.py
!COMMANDLINE_ARGS="--exit"  python launch.py

#!pip install --upgrade fastapi==0.90.1

if gMode == GDriveSaveMode.Models_only:
  # Use config file in google drive
  if not os.path.exists(output_path + '/ui-config.json'):
    print("Create new ui-config.json file.")    
    !wget https://github.com/sagiodev/stablediffusion_webui/raw/master/ui-config.json -O {output_path + '/ui-config.json'}
  if not os.path.exists(output_path + '/config.json'):
    print("Create new config.json file.")    
    !wget https://github.com/sagiodev/stablediffusion_webui/raw/master/config.json -O {output_path + '/config.json'}

  #!rm /content/stable-diffusion-webui/ui-config.json
  !ln -s {output_path + '/ui-config.json'} {root}/stable-diffusion-webui/
  !ln -s {output_path + '/config.json'} {root}/stable-diffusion-webui/
  !ln -s {output_path}/outputs
  
  # embeddings folder on Google Drive
  !mkdir -p {output_path}/embeddings  
  !rm -rf embeddings
  !ln -s {output_path}/embeddings

  # save parameter file in google drive
  if not os.path.exists(output_path + '/params.txt'):
    !touch {output_path + '/params.txt'}
  !ln -s {output_path}/params.txt

  # link all models in the models folder
  %cd {root}/stable-diffusion-webui/models/Stable-diffusion
  models_in_google_drive = glob.glob(output_path + '/models/*')
  print('Models in Google Drive: %s'%models_in_google_drive)
  for f in models_in_google_drive:
    !ln -s {f}

  # link all upscalers in the model folder
  !mkdir -p {root}/stable-diffusion-webui/models/ESRGAN
  %cd {root}/stable-diffusion-webui/models/ESRGAN
  upscalers_in_google_drive = glob.glob(output_path + '/ESRGAN/*')
  print('Upscalers in Google Drive: %s'%upscalers_in_google_drive)
  for f in upscalers_in_google_drive:
    !ln -s {f}

  # use lora model folder in google drive
  !mkdir -p {output_path}/Lora
  %cd {root}/stable-diffusion-webui/models
  !rm -rf Lora
  !ln -s {output_path}/Lora

  # use hypernetwork folder in google drive
  %cd {root}/stable-diffusion-webui/models
  !rm -rf hypernetworks
  !ln -s {output_path}/hypernetworks


if ControlNet:
  print("Installing ControlNet extension...")
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/Mikubill/sd-webui-controlnet.git
  %cd {root}/stable-diffusion-webui/extensions/sd-webui-controlnet
  !pip install -r requirements.txt
  
  %cd {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/models
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors -d {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_canny-fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors -d {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_depth-fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors -d {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_hed-fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors -d {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_mlsd-fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors -d {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_normal-fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors -d {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_openpose-fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors -d {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_scribble-fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors -d {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_seg-fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/hand_pose_model.pth -d {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/openpose -o hand_pose_model.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/body_pose_model.pth -d {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/openpose -o body_pose_model.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/dpt_hybrid-midas-501f0c75.pt -d {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/midas -o dpt_hybrid-midas-501f0c75.pt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_large_512_fp32.pth -d {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/mlsd -o mlsd_large_512_fp32.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_tiny_512_fp32.pth -d {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/mlsd -o mlsd_tiny_512_fp32.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/network-bsds500.pth -d {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/hed -o network-bsds500.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/upernet_global_small.pth -d {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/uniformer -o upernet_global_small.pth
  print("ControlNet install completed.")

# clear output
if Clear_Log:
  clear()

if gMode == GDriveSaveMode.Models_only:
  print('Models in Google Drive: %s'%models_in_google_drive)
  print('Upscalers in Google Drive: %s'%upscalers_in_google_drive)

!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' {root}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' {root}/stable-diffusion-webui/launch.py
!cd {root}/stable-diffusion-webui/repositories/k-diffusion; git config core.filemode false
%cd {root}/stable-diffusion-webui  
args = f'--gradio-img2img-tool color-sketch --enable-insecure-extension-access --gradio-queue --xformers'

if NGROK:
  args += f' --ngrok {NGROK} '
else:
  args += ' --share '
if username and password:
  args += f' --gradio-auth {username}:{password} '
print(f'WEBUI ARGUMENTS: {args}')
!python {root}/stable-diffusion-webui/launch.py {args}